In [1]:
import numpy as np
import pandas as p
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib
import scipy
from scipy import stats
# from adjustText import adjust_text
from Bio import SeqIO
from Bio.Seq import Seq
%matplotlib inline

import copy


%load_ext autoreload
%autoreload 2

# import gzip
# from cyvcf2 import VCF

from matplotlib import gridspec
from matplotlib.gridspec import GridSpec

from itertools import combinations
from itertools import combinations_with_replacement
from itertools import permutations
from ast import literal_eval

import sys
import os

home_dir = '~/Documents/Stanford/Research/EvolvingFront/'
home_dir = os.path.expanduser(home_dir)

tools_path = f'{home_dir}/code/processing/tools/tools.py'
sys.path.append(os.path.dirname(os.path.expanduser(tools_path)))
import tools as tools

sns.set_style('white')
sns.set_style('ticks')
sns.set_color_codes()



We observe that there are many more mutatations that improve respiration performance than improve fermentation performance in our experiments. While differences in mutation accessibility could be driving these differences, another difference could be a shift in the relative amount of each performance in the evolution condition. In particular, the parental strains with first-step mutations likely ferment faster than the ancestor, reducing the amount of fermentation phase and therefore reducing its influence on the fitness of mutants in this environment. 

To test if this is the case, we can compare mutants with similar fitnesses in the fitness remeasurments though achieving this by different means (either only respiration or both fermentation and respiration) and compare their fitnesses in the evolution condition.

In [2]:
merged_fitness = p.read_csv(f'{home_dir}/data/fitness_withMutations.csv',index_col=0)

conditions = ['Evo2D_Ira1_Mis_1','Evo2D_Ira1_Mis_2']

evolution_fitness = {}

for condition in conditions:
    evolution_fitness[condition] = p.read_csv(f'{home_dir}/EvolutionTracking/FitMut2_processing/out_fitmut1_thresholded/{condition}_MutSeq_Result.csv')

In [3]:
evolution_fitness[conditions[0]]

,Fitness,Establishment_Time,Error_Fitness,Error_Establishment_Time,Likelihood_Log,Likelihood_Log_Adaptive,Likelihood_Log_Neutral,Mean_Fitness,Kappa_Value,Mutant_Cell_Fraction,Inference_Time
0,0.0,0.0,0.0,0.0,-7.760886,-34.757836,-26.996949,0.000010,2.000000,0.001695,875.12577
1,0.0,0.0,0.0,0.0,-8.341183,-31.764848,-23.423664,0.000822,1.543212,0.005608,NaN
2,0.0,0.0,0.0,0.0,-9.218625,-30.459707,-21.241083,0.002220,1.591152,0.014184,NaN
3,0.0,0.0,0.0,0.0,-9.361807,-32.089101,-22.727294,0.015797,1.232281,0.051173,NaN
4,0.0,0.0,0.0,0.0,-10.980815,-29.731338,-18.750524,0.030783,1.027027,0.134946,NaN
...,...,...,...,...,...,...,...,...,...,...,...
70906,0.0,0.0,0.0,0.0,-18.178500,-30.453238,-12.274737,NaN,NaN,NaN,NaN
70907,0.0,0.0,0.0,0.0,-17.931795,-29.504503,-11.572708,NaN,NaN,NaN,NaN
70908,0.0,0.0,0.0,0.0,-17.805101,-32.010630,-14.205529,NaN,NaN,NaN,NaN
70909,0.0,0.0,0.0,0.0,-17.841348,-61.292565,-43.451217,NaN,NaN,NaN,NaN


In [4]:
p.read_csv(f'{home_dir}/EvolutionTracking/FitMut2_processing/data_thresholded/{condition}_counts.csv')

,99.0,44.0,148.0,112.0,72.0,31.0,6.0,1.0
0,17.0,9.0,24.0,20.0,25.0,6.0,2.0,1.0
1,24.0,20.0,40.0,50.0,30.0,12.0,2.0,0.0
2,19.0,14.0,63.0,32.0,27.0,14.0,1.0,0.0
3,2.0,6.0,9.0,4.0,1.0,2.0,0.0,0.0
4,10.0,3.0,12.0,4.0,5.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...
70905,0.0,0.0,0.0,0.0,12.0,137.0,285.0,285.0
70906,0.0,0.0,0.0,0.0,1.0,3.0,16.0,17.0
70907,0.0,0.0,0.0,0.0,3.0,250.0,252.0,388.0
70908,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0
